# fft score cam implementation


### Imports

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import wandb
import pickle
#from PIL import Image
import torch.nn.functional as F
from torch.utils.data import DataLoader
from skimage import transform
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import cv2
from plotting_utils import RF_Visualizer
import pandas as pd
from scipy.fft import fft, ifft,fftshift
import os
import math
from scipy import signal

cur_dir = os.getcwd()
print(cur_dir)



### Load Data

In [ ]:
if __name__ == "__main__":
    from model import Baseline_CNN1D
    from dataset_tprime import TPrimeDataset
    import argparse
    from scipy.io import loadmat


    # seqs = {}
    # with open("/home/sagetrudeau/Datasets/ORACLE/mat_files/raw/train.pkl", "rb") as f:#Oracle
    # #with open("/raid/backup_storage_oldDGX/LORA/Year_1_outdoor/outdoor_dataset_1/mat_files/raw/train.pkl", "rb") as f:
    #     file_dict = pickle.load(f)
    # filenames = file_dict["files"]
    # print(filenames)
    # for i, name in enumerate(filenames):
    #     #print(i, len(filenames))
    #     seqs[name] = loadmat(name)["f_sig"][0]
    # #dataset = InDistributionTestDataset(seqs,"val")
    # dataset = InDistributionTestDatasetContinuous(seqs, "val")
    # #dataset = InDistributionTrainDataset(seqs)
    # train_dataset = InDistributionTrainDataset(seqs)
    # del seqs
    args=argparse.Namespace()
    args.protocols = ['802_11ax', '802_11b_upsampled', '802_11n', '802_11g']
    args.noise = True
    args.snr_db = [30]
    args.raw_path = "/home/sagetrudeau/Projects/t-prime/data/DATASET1_1"
    args.slicelen = 512
    args.overlap_ratio = 0.0
    args.postfix = ''
    args.raw_data_ratio = 1.0
    args.channel = None
    args.out_mode = 'real'
    args.worker_batch_size = 512

    ds_test = TPrimeDataset(args.protocols,
                          ds_path=args.raw_path,
                          ds_type='test',
                          snr_dbs=args.snr_db,
                          slice_len=args.slicelen,
                          slice_overlap_ratio=float(args.overlap_ratio),
                          raw_data_ratio=args.raw_data_ratio,
                          file_postfix=args.postfix,
                          override_gen_map=False,    # it will use the same as above call
                          apply_wchannel=args.channel,
                          apply_noise=args.noise,
                          out_mode=args.out_mode)
    dataloader = DataLoader(ds_test, batch_size=args.worker_batch_size, shuffle=False)


### Load model


In [ ]:

#t-prime
PATH = './results/t-prime/SNR30/'
Nclass = 4
num_channels = 2
num_feats = 1
slice_len = 512

device = torch.device('cpu')
print(device)
model = Baseline_CNN1D(classes=Nclass, numChannels=num_channels, slice_len=slice_len)
checkpoint = torch.load(cur_dir+"/model/model.baseline_cnn1d.nochan.range.pt")
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device) # reload the model on the appropriate device
model.device = device
model.eval()    # set the evaluation mode
print(model,next(model.parameters()).device)
print(model.device)




### Experimenting with TSNE

In [ ]:
from sklearn.manifold import TSNE
import itertools
import cuml
# Function to extract features from the last fully connected layer before softmax
def extract_features(model, data_loader):
    features = []
    labels = []
    samples_to_run=5
    i=0
    with torch.no_grad():
        for data, target in data_loader:
            # Ensure data is converted to float and moved to the correct device
            data = data.float().to(model.device)  # Convert data to float here
            # Get the output from the last fully connected layer (fc1)
            x = model.conv1(data)
            x = model.relu1(x)
            x = model.maxpool1(x)
            x = model.conv2(x)
            x = model.relu2(x)
            x = model.maxpool2(x)
            x = torch.flatten(x, 1)
            x = model.fc1(x)
            x = model.relu3(x)
            # x= model.fc2(x)
            # x=model.logSoftmax(x)
            features.append(x.cpu().numpy())
            labels.append(target.numpy())
            if i > samples_to_run:
                break
            i+=1
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels

features, labels = extract_features(model, dataloader)
print(features.shape,labels.shape)

fig, ax = plt.subplots(1, 3, figsize=(20,7), constrained_layout=True)
for c, per in zip(itertools.count(), [5, 30, 50]):
    tsne = cuml.manifold.TSNE(n_components=2,
                perplexity=per,
                n_neighbors=per*4)
    tsne = tsne.fit_transform(features)
    scatter = ax[c].scatter(tsne[:-1, 0], tsne[:-1, 1], c=labels[:-1], cmap='tab10', s=0.3)
    scatter = ax[c].scatter(tsne[-1, 0], tsne[-1, 1], c=labels[-1], cmap='tab10',marker='*', edgecolors='k', s=50)
    ax[c].set_title(f'Perplexity: {per}', fontsize=16)    

fig.suptitle('t-SNE Dimensionality reduction', fontweight='bold', fontsize=25)
cbar = fig.colorbar(scatter, boundaries=np.arange(11)-0.5, location='right')
cbar.set_ticks(np.arange(10))
cbar.set_ticklabels(np.arange(10))
plt.show()

In [ ]:


# Function to augment the last sample in the dataloader by replacing it with noise of equal power
def augment_sample(model, dataloader):
    # Access the dataset directly from the dataloader
    dataset = dataloader.dataset
    print(type(dataset))
    last_index = len(dataset) - 1
    last_index = 10000

    # Extract the last input and label
    inputs, labels = dataset[last_index]
    aug_inputs = inputs.copy()

    # Print the shape and values of the inputs
    print("Original numpy array values:")
    print(inputs)
    print(f"Shape of inputs: {inputs.shape}")

    # Convert numpy array to torch tensor
    slice_of_trans = torch.from_numpy(inputs).float()

    # Check the device of the model
    device = next(model.parameters()).device
    print(f"Model is on device: {device}")

    # Move the tensor to the same device as the model
    slice_of_trans = slice_of_trans.to(device)

    # Ensure the input shape matches the model's expected dimensions
    if len(slice_of_trans.shape) == 2:
        slice_of_trans = slice_of_trans.unsqueeze(0)
    print("og",slice_of_trans)
    # Run the model prediction
    original_outputs = model(slice_of_trans).detach().cpu().numpy()
    print(f"Original outputs shape: {original_outputs.shape}")
    print(f"Original prediction: {original_outputs}")
    print(f"True label: {labels}")

    # Grab the slice and calculate the power of the original signal
    complex_slice = inputs[0] + 1j * inputs[1]
    total_power_original = np.sum(np.abs(complex_slice)**2)
    #total_power_original = 513.93585
    print("power target",total_power_original)

    # Generate a noise array with the same length as the original sample
    slice_len = complex_slice.shape
    noise_array = np.random.normal(size=slice_len) + 1j * np.random.normal(size=slice_len)
    total_power_noise = np.sum(np.abs(noise_array)**2)

    # Scale the noise array to match the total power of the original signal
    scaling_factor = np.sqrt(total_power_original / total_power_noise)
    equal_power_noise = noise_array * scaling_factor

    # Replace the last sample with the equal power noise
    real_part = np.real(equal_power_noise)
    imaginary_part = np.imag(equal_power_noise)
    # Update the inputs with the modified slice
    aug_inputs[0]= real_part
    aug_inputs[1] = imaginary_part

    # aug_slice_of_trans = torch.from_numpy([real_part,imaginary_part]).float()
    # aug_slice_of_trans = aug_slice_of_trans.to(device)
  

    # Run predictions after augmentation for verification
    aug_slice_of_trans = torch.from_numpy(aug_inputs).float()
    aug_slice_of_trans = aug_slice_of_trans.to(device)
    if len(aug_slice_of_trans.shape) == 2:
        aug_slice_of_trans = aug_slice_of_trans.unsqueeze(0)
    print("augmented",aug_slice_of_trans)
    augmented_outputs = model(aug_slice_of_trans).detach().cpu().numpy()
    original_prediction = np.argmax(original_outputs, axis=1)[-1]
    augmented_prediction = np.argmax(augmented_outputs, axis=1)[-1]
    # Print the shape and values of the inputs
    print("Augmented numpy array values:")
    print(aug_inputs)
    print(f"Shape of inputs: {aug_inputs.shape}")

    print(f"Augmented output: {augmented_outputs}")
    print("Original prediction:", original_prediction)
    print("Augmented prediction:", augmented_prediction)

    # Update the dataset with the modified 
    #dataset = list(dataset)
    
    dataset.augment_signal_cache_item(last_index, aug_inputs)
    check_input, label = dataset[last_index]
    print(check_input)
    # Create a new dataloader with the modified dataset
    #new_dataloader = DataLoader(dataset, batch_size=dataloader.batch_size, shuffle=False)
    
    return dataloader

# Function to plot t-SNE with and without augmentation
def plot_tsne_with_augmentation(model, dataloader):
    # Extract features and labels from the original data
    features, labels = extract_features(model, dataloader)
    print("Original features shape:", features.shape, "Labels shape:", labels.shape)

    # Augment the last sample in the dataloader
    augmented_dataloader = augment_sample(model,dataloader)

    # Extract features and labels from the augmented data
    augmented_features, augmented_labels = extract_features(model, augmented_dataloader)
    print("Augmented features shape:", augmented_features.shape, "Labels shape:", augmented_labels.shape)

    fig, ax = plt.subplots(2, 3, figsize=(20, 14), constrained_layout=True)
    
    for c, per in zip(itertools.count(), [5, 30, 50]):
        # Original t-SNE
        tsne = cuml.manifold.TSNE(n_components=2, perplexity=per, n_neighbors=per*4)
        tsne_result = tsne.fit_transform(features)
        scatter = ax[0, c].scatter(tsne_result[:-1, 0], tsne_result[:-1, 1], c=labels[:-1], cmap='tab10', s=0.3)
        scatter = ax[0, c].scatter(tsne_result[-1, 0], tsne_result[-1, 1], c=labels[-1], cmap='tab10', marker='*', edgecolors='k', s=50)
        ax[0, c].set_title(f'Original - Perplexity: {per}', fontsize=16)
        
        # Augmented t-SNE
        tsne_augmented = cuml.manifold.TSNE(n_components=2, perplexity=per, n_neighbors=per*4)
        tsne_augmented_result = tsne_augmented.fit_transform(augmented_features)
        scatter = ax[1, c].scatter(tsne_augmented_result[:-1, 0], tsne_augmented_result[:-1, 1], c=augmented_labels[:-1], cmap='tab10', s=0.3)
        scatter = ax[1, c].scatter(tsne_augmented_result[-1, 0], tsne_augmented_result[-1, 1], c=augmented_labels[-1], cmap='tab10', marker='*', edgecolors='k', s=50)
        ax[1, c].set_title(f'Augmented - Perplexity: {per}', fontsize=16)
    
    fig.suptitle('t-SNE Dimensionality Reduction Before and After Augmentation', fontweight='bold', fontsize=25)
    cbar = fig.colorbar(scatter, boundaries=np.arange(11)-0.5, location='right')
    cbar.set_ticks(np.arange(10))
    cbar.set_ticklabels(np.arange(10))
    plt.show()


plot_tsne_with_augmentation(model, dataloader)
